# Polish police statistics

This script collects data from Polish police website: http://slaska.policja.gov.pl/kat/form/23,Statystyki.html.

## Web Scraping

In [3]:
from urllib.request import urlopen
from urllib.error import HTTPError
from bs4 import BeautifulSoup
import csv

URL = 'http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page='
RANGE = 93
CSV = '../data/slask-police.csv'

header = []
rows = []

def scrape_page(html):
    tbody = BeautifulSoup(html.read(), "html.parser").tbody
    for tr in tbody.findAll('tr'):
        if tr.th:
            if not header:
                for th in tr.findAll('th'):
                    header.append(th.get_text())
        if tr.td:
            row = []
            for td in tr.findAll('td'):
                row.append(td.get_text())
            rows.append(row)

for i in range(RANGE):
    try:
        url = URL + str(i)
        print('Open page', url)
        html = urlopen(url)
    except HTTPError as e:
        print(e)
    else:
        scrape_page(html)
     
rows.insert(0, header)

with open(CSV, "w") as f:
    writer = csv.writer(f)
    writer.writerows(rows)

Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=0
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=1
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=2
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=3
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=4
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=5
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=6
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=7
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=8
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=9
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=10
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=11
Open page http://slaska.policja.gov.pl/kat/form/23,Statystyki.html?page=12
Open page http://slaska.policja.gov

# Check csv file

In [4]:
import pandas as pd
df1 = pd.read_csv(CSV, parse_dates=['Data'], dayfirst=True, index_col='Data')
df1

,Rozboje,Bójki i pobicia,Kradzieże z włamaniem,Kradzieże,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,Nietrzeźwi kierujący,Kolizje drogowe
Data,,,,,,,,,
2016-03-28,1,6,21.0,8,3,0,4,16,53
2016-03-27,2,1,11.0,9,2,0,4,17,42
2016-03-26,2,2,13.0,15,10,0,12,21,117
2016-03-25,3,3,24.0,30,4,1,3,19,142
2016-03-24,2,2,26.0,53,6,0,8,18,142
2016-03-23,0,0,18.0,36,5,0,10,17,162
2016-03-22,1,0,25.0,46,16,3,16,17,178
2016-03-20,5,3,18.0,23,1,0,2,28,77
2016-03-21,2,2,37.0,47,3,0,4,23,131


# Looking for duplicates

We can drop duplicates:

In [5]:
df2 = pd.read_csv(CSV, parse_dates=['Data'], dayfirst=True)
df3 = df2.drop_duplicates('Data', keep='last').set_index('Data') #done
print(len(df2), len(df3))

2776 2767


Or we can list all rows witch are duplicated:

In [6]:
df2['is_duplicated'] = df2.duplicated('Data', keep=False)
df2[(df2["is_duplicated"] == True)]

,Data,Rozboje,Bójki i pobicia,Kradzieże z włamaniem,Kradzieże,Wypadki drogowe,Zabici w wypadkach,Ranni w wypadkach,Nietrzeźwi kierujący,Kolizje drogowe,is_duplicated
32,2016-02-25,2,3,22.0,49,6,0,7,11,174,True
39,2016-02-18,2,1,26.0,52,3,1,2,13,154,True
40,2016-02-18,3,1,27.0,40,1,0,1,12,93,True
163,2015-10-11,3,4,20.0,27,3,0,4,36,65,True
164,2015-10-11,2,1,25.0,44,9,0,15,19,134,True
175,2015-09-28,4,1,23.0,51,10,1,11,13,143,True
176,2015-09-28,5,3,47.0,58,12,1,13,23,147,True
187,2015-09-17,1,5,35.0,48,10,1,8,23,191,True
188,2015-09-17,1,1,19.0,56,4,0,4,26,143,True
706,2014-03-28,1,5,36.0,59,12,2,15,16,139,True
